# Recommendation System Via Hybrid Comparison  
#### <span style="color: cornflowerblue">Team 01 | CSPB 4502 | 11/16/22</span>

The following is a two-stage filtering of raw data for the Netflix recommendation competition to retain only users sufficiently similar to the randomly chosen subscriber (represented by *user_id*). The first stage filters by overlap of movies rated. The second filters by correlation of ratings.

The threshold for the first filter is a percentage of movies rated by *user_id* that another user has rated.

The threshold for the second filter is a correlation coefficient between the ratings of *user_id* and a user for only movies they both have rated.

Threshold selections can be found at the top of the program.

In [1]:
import pandas as pd
import numpy as np

# csv of combined netflix ratings (see EDA_Netflix on how this was created)
rating_filepath = r'../data/ratings.csv'

# seed for randomization
seed = 3
np.random.seed(seed)

movie_thresh = 0.6    # threshold proportion of random_ID's movies that a user must have rated to be retained
rho_thresh = 0.4   # threshold a user's correlation coefficient must meet to be retained

In [2]:
# runtime: 20 sec
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv(rating_filepath, index_col=0)

In [3]:
df.head(10)

,index,Movie_Id,CustomerID,Rating,Date
0,79334307,14358,6,2,2005-12-04
1,34092597,6134,6,4,2005-01-12
2,32652672,5926,6,4,2005-10-26
3,38173947,6797,6,3,2004-11-10
4,20618167,3905,6,3,2005-12-04
5,99601160,17560,6,3,2004-11-10
6,75553656,13651,6,3,2004-11-13
7,40752419,7230,6,5,2005-08-29
8,21282866,3962,6,4,2005-08-29
9,69300001,12560,6,3,2005-12-04


In [4]:
len(df)
# Sanity check: total number of ratings before any thresholds applied

100480507

In [5]:
max_value = len(df["CustomerID"].unique())
max_value
# Output: total number of users in data frame (highest ID number)

480189

In [6]:
# runtime: 3 sec
# choose randomly a user for whom to give recommendations
random_id = int(df["CustomerID"].sample(1))


##################################################
####### Randomly selected user's ID number #######
##################################################

In [7]:
# Pulling off the records of just the target user
user_df = df[df["CustomerID"] == random_id]
user_df.head(10)
len(user_df)

#################################################
##### Number of movies rated by random_user #####
#################################################

156

In [8]:
# Putting the movies watched by random_user into a list
movies_watched = user_df["Movie_Id"].tolist()

# Check: format, sanity check
print(movies_watched[0:10])


[12799, 14606, 6037, 12299, 10906, 16604, 7624, 6134, 17169, 4972]


In [9]:
# Runtime: 2 seconds
movies_watched_df = df[df["Movie_Id"].isin(movies_watched)]


# Output: all rows referring to a movie random_id rated
movies_watched_df.head(20)

,index,Movie_Id,CustomerID,Rating,Date
1,34092597,6134,6,4,2005-01-12
6,75553656,13651,6,3,2004-11-13
8,21282866,3962,6,4,2005-08-29
24,91333971,16242,6,3,2005-04-20
31,81114956,14574,6,4,2004-09-27
35,29531350,5401,6,2,2004-12-14
37,92320246,16377,6,4,2004-10-14
49,62315841,11314,6,3,2005-12-04
50,67431159,12293,6,5,2004-09-15
53,85766546,15205,6,3,2004-10-19


In [10]:
user_movie_count = movies_watched_df.groupby(["CustomerID"]).Movie_Id.count()
user_movie_count.head(10)
# Output: number of moviews a user has rated that randome_user has also rated
# Check: Most users will have rated at least one of random_user's rated movies

CustomerID
6      92
7     126
8      31
10     47
25      8
33      7
42     28
59     32
79    113
83      2
Name: Movie_Id, dtype: int64

In [11]:
############################################
####### Overlap threshold applied here #####
############################################

# We will pull off only users who have rated at least this proportion of the movies the target user has rated.
m_count = movie_thresh*len(movies_watched)
m_count

# Output: movie count threshold for retention of a user

93.6

In [12]:
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["CustomerID", "movie_count"]

# Choosing 50% instead of 60% makes a huge difference in number of records that finally get through.

# choose a ratio of 0.50. User ids who watched at least 50 percent of target user's movies 
users_same_movies = user_movie_count[user_movie_count["movie_count"] > m_count].sort_values("movie_count", ascending=False)
users_same_movies.nunique()

# Output: total number of users that rated at least 50% of movies rated by random_user

CustomerID     24331
movie_count       63
dtype: int64

In [13]:
# Sanity check: movie_count should be no higher than len(movies_watched) above
users_same_movies.head(10)

,CustomerID,movie_count
372468,2118461,156
126523,716173,156
68399,387418,156
93724,531050,156
458343,2606799,156
231665,1314869,155
428897,2439493,155
412947,2349412,155
446204,2537543,154
292899,1664010,154


In [14]:
similar_users = users_same_movies["CustomerID"].tolist()
similar_users_df = df[df["CustomerID"].isin(similar_users)]

# Sanity check: ALL ratings of users heavily overlapping random_id's movies
similar_users_df.sort_values(by=["Movie_Id", "CustomerID"]).head(20)

,index,Movie_Id,CustomerID,Rating,Date
111909,275,1,3321,3,2005-09-27
559812,186,1,14756,4,2005-12-27
621248,311,1,16272,4,2005-01-20
1138129,141,1,30245,5,2004-10-19
1162138,3,1,30878,4,2005-12-26
3262137,264,1,87113,2,2005-09-21
3711703,125,1,99400,5,2005-09-16
4593359,238,1,122197,1,2005-03-09
5046855,252,1,134001,4,2005-05-06
5459036,454,1,145873,3,2004-02-25


In [15]:
# Number of ratings by users that have strong movie overlap with random_id
# Compare to len(df) above (about 1/4?)
len(similar_users_df)

24682064

In [16]:
# Create CustomerID vs Movie_Id pivot talbe so that
# corr() can find pairwise correlation between columns.

# Checking the starting format 
print(user_df.head(10))
del(user_df)

             index  Movie_Id  CustomerID  Rating        Date
20236871  70629382     12799      531050       4  2005-10-22
20236872  81390529     14606      531050       3  2005-10-22
20236873  33426000      6037      531050       2  2005-10-22
20236874  67536946     12299      531050       4  2005-10-22
20236875  59319689     10906      531050       3  2005-12-02
20236876  93599680     16604      531050       5  2005-10-22
20236877  43164086      7624      531050       3  2005-10-22
20236878  34082489      6134      531050       5  2005-12-02
20236879  97063927     17169      531050       5  2005-10-22
20236880  26764256      4972      531050       3  2005-12-02


In [17]:
# Sanity check: movie_watched_df should include all of random_id's records
movies_watched_df[movies_watched_df["CustomerID"]==random_id].sort_values(by=["Movie_Id", "CustomerID"])

,index,Movie_Id,CustomerID,Rating,Date
20236911,787820,191,531050,5,2005-10-22
20237011,1635102,313,531050,4,2005-10-22
20236953,4555085,886,531050,5,2005-12-02
20236962,5540915,1110,531050,3,2005-10-22
20236903,6058522,1180,531050,3,2005-10-22
20237020,6718168,1307,531050,4,2005-10-22
20236884,7175463,1428,531050,4,2005-10-22
20237018,7683067,1542,531050,5,2005-10-22
20236983,8455111,1692,531050,4,2005-10-22
20236954,9207305,1810,531050,4,2005-10-22


In [18]:
# Create a Movie_Id vs. CustomerID pivot table
# Running time: immediate
movies_watched_df2 = movies_watched_df[movies_watched_df['CustomerID'].isin(similar_users)]
movies_watched_df3 = movies_watched_df2.drop(['Date'], axis=1)
movies_watched_df_pivot = movies_watched_df3.pivot(index=['CustomerID'], columns=['Movie_Id'], values="Rating")

In [19]:
# Check: No more than about 40% of ratings are NaN
movies_watched_df_pivot.head(10)

Movie_Id,191,313,886,1110,1180,1307,1428,1542,1692,1810,1905,2128,2152,2200,2782,2913,3079,3290,3333,3624,3638,3825,3864,3938,3962,4123,4216,4266,4306,4356,4432,4545,4640,4683,4951,4972,4996,5087,5239,5293,5327,5401,5496,5549,5561,5582,5775,5807,6037,6042,6134,6243,6386,6428,6615,6829,6859,7155,7234,7364,7509,7513,7624,7635,8467,8596,8764,8832,8851,9111,9340,9528,9617,9628,9756,9818,10042,10170,10344,10358,10378,10379,10436,10491,10550,10583,10607,10664,10906,10947,11040,11089,11182,11283,11314,11607,11781,11812,12052,12293,12299,12317,12470,12494,12582,12672,12785,12799,12904,13081,13622,13651,13673,13728,14103,14146,14313,14407,14410,14538,14550,14574,14606,14621,14691,14718,15107,15116,15125,15156,15205,15393,15471,15472,15509,15788,15844,15894,15952,15975,16128,16242,16265,16377,16384,16552,16604,16872,16879,16954,17002,17157,17169,17215,17324,17355
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,4.0,5.0,4.0,NaN,5.0,5.0,4.0,4.0,NaN,2.0,5.0,4.0,3.0,3.0,5.0,NaN,NaN,5.0,NaN,5.0,5.0,5.0,NaN,4.0,5.0,4.0,3.0,5.0,5.0,4.0,5.0,5.0,5.0,3.0,4.0,3.0,3.0,NaN,NaN,4.0,NaN,3.0,4.0,5.0,4.0,5.0,5.0,5.0,3.0,3.0,5.0,4.0,3.0,5.0,NaN,4.0,3.0,5.0,5.0,5.0,4.0,NaN,3.0,3.0,NaN,5.0,4.0,4.0,NaN,4.0,3.0,5.0,5.0,5.0,4.0,4.0,5.0,2.0,3.0,3.0,NaN,NaN,3.0,5.0,5.0,3.0,5.0,NaN,3.0,5.0,5.0,4.0,5.0,5.0,4.0,5.0,4.0,NaN,5.0,5.0,3.0,NaN,4.0,NaN,5.0,3.0,3.0,NaN,5.0,4.0,5.0,3.0,5.0,5.0,NaN,4.0,4.0,NaN,4.0,3.0,5.0,4.0,4.0,NaN,5.0,2.0,5.0,4.0,NaN,4.0,4.0,NaN,4.0,4.0,NaN,4.0,3.0,4.0,NaN,NaN,4.0,2.0,5.0,5.0,4.0,5.0,4.0,NaN,4.0,5.0,NaN,4.0,4.0,4.0,5.0,5.0
79,4.0,NaN,5.0,4.0,4.0,3.0,3.0,4.0,NaN,3.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,NaN,NaN,1.0,3.0,3.0,5.0,5.0,5.0,4.0,NaN,NaN,5.0,4.0,4.0,NaN,5.0,3.0,NaN,3.0,5.0,5.0,5.0,4.0,2.0,5.0,3.0,NaN,NaN,4.0,NaN,5.0,4.0,5.0,3.0,NaN,4.0,NaN,4.0,4.0,4.0,4.0,2.0,NaN,4.0,NaN,4.0,4.0,3.0,NaN,3.0,NaN,NaN,4.0,4.0,NaN,4.0,5.0,3.0,NaN,4.0,NaN,4.0,3.0,3.0,1.0,NaN,5.0,NaN,1.0,4.0,NaN,4.0,5.0,NaN,4.0,3.0,5.0,4.0,NaN,4.0,5.0,NaN,4.0,5.0,3.0,4.0,5.0,NaN,4.0,4.0,4.0,5.0,4.0,NaN,3.0,NaN,4.0,NaN,NaN,4.0,NaN,4.0,3.0,NaN,4.0,NaN,5.0,4.0,3.0,3.0,5.0,NaN,NaN,3.0,3.0,2.0,3.0,NaN,1.0,NaN,NaN,4.0,5.0,4.0,NaN,4.0,4.0,3.0,5.0,3.0,5.0,4.0,4.0,NaN,5.0,5.0,NaN,4.0,2.0
134,5.0,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NaN,5.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,5.0,5.0,NaN,5.0,5.0,NaN,5.0,5.0,NaN,NaN,5.0,NaN,NaN,5.0,3.0,5.0,NaN,5.0,5.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,NaN,5.0,5.0,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,5.0,5.0,NaN,5.0,4.0,5.0,5.0,NaN,5.0,5.0,5.0,NaN,NaN,5.0,NaN,5.0,5.0,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,5.0,5.0,NaN,5.0,NaN,5.0,5.0,5.0,5.0,NaN,NaN,5.0,NaN,5.0,NaN,NaN,5.0,5.0,5.0,5.0,4.0,5.0,NaN,NaN,NaN,5.0,4.0,5.0,5.0,NaN,4.0,5.0,5.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0,5.0,5.0,5.0,5.0,5.0,NaN,5.0
188,3.0,2.0,4.0,3.0,3.0,3.0,4.0,2.0,NaN,4.0,4.0,4.0,NaN,3.0,5.0,NaN,3.0,4.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,NaN,2.0,4.0,3.0,4.0,4.0,3.0,3.0,3.0,NaN,3.0,4.0,3.0,4.0,NaN,3.0,3.0,NaN,NaN,4.0,3.0,3.0,4.0,3.0,4.0,3.0,NaN,NaN,3.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,3.0,4.0,NaN,3.0,4.0,NaN,4.0,NaN,3.0,NaN,NaN,3.0,3.0,NaN,4.0,4.0,NaN,3.0,NaN,NaN,4.0,NaN,4.0,3.0,5.0,3.0,NaN,4.0,3.0,4.0,5.0,4.0,4.0,4.0,3.0,NaN,NaN,5.0,NaN,4.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,3.0,NaN,4.0,4.0,5.0,3.0,NaN,5.0,3.0,4.0,3.0,5.0,NaN,NaN,4.0,3.0,2.0,4.0,4.0,NaN,NaN,3.0,3.0,4.0,4.0,NaN,NaN,3.0,NaN,2.0,NaN,4.0,4.0,4.0,4.0,5.0,NaN,4.0,4.0,3.0,4.0,NaN,5.0,NaN,NaN,3.0,3.0
199,4.0,4.0,5.0,NaN,4.0,4.0,4.0,3.0,NaN,4.0,4.0,4.0,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,5.0,5.0,3.0,5.0,NaN,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,5.0,5.0,NaN,NaN,4.0,NaN,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN,5.0,4.0,4.0,4.0,NaN,4.0,4.0,4.0,5.0,3.0,4.0,5.0,3.0,4.0,5.0,3.0,NaN,NaN,4.0,5.0,4.0,4.0,4.0,NaN,4.0,4.0,5.0,3.0,4.0,NaN,4.0,NaN,NaN,3.0,NaN,NaN,NaN,3.0,5.0,5.0,NaN,4.0,5.0,4.0,4.0,5.0,3.0,2.0,3.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,NaN,NaN,4.

In [20]:
# Pivot, to prepare to correlate columns
corr_df = movies_watched_df_pivot.transpose()


In [21]:
# Data cleaning: corrwith ignores NaN
corr_df2 = corr_df.corrwith(corr_df[random_id], method='pearson')
# running time: 2 secs

In [22]:
# Unsorted
corr_df2.head(10)

CustomerID
7       0.413260
79      0.109080
134     0.194939
188     0.226901
199     0.261744
602     0.068084
1333    0.298962
1442   -0.002107
1457    0.173323
1500    0.310531
dtype: float64

In [23]:
# Sanity check: Count of unique users with sufficient 
# overlap matches length of user_same_movies above
corr_df3 = corr_df2.sort_values(ascending=False).drop_duplicates()
corr_df3.shape

(24324,)

In [24]:
# Sanity check: only random_id has correlation 
# coefficient of 1; only random_id with small
# count of ratings would have correlation
# coefficients approaching 0.8
corr_df3.head(10)

CustomerID
531050     1.000000
1180814    0.589639
1726783    0.565858
31387      0.546136
2444333    0.517946
1688893    0.508165
1500471    0.507394
1158943    0.505248
1683765    0.505033
1352424    0.504831
dtype: float64

In [25]:
# Uninteresting formatting
# Convert series to dataframe
corr_df4 = corr_df3.to_frame()


In [26]:
# Check: formatting
corr_df4.rename(columns={0:'corr'}, inplace=True )
corr_df4.head(10)

,corr
CustomerID,
531050,1.000000
1180814,0.589639
1726783,0.565858
31387,0.546136
2444333,0.517946
1688893,0.508165
1500471,0.507394
1158943,0.505248
1683765,0.505033


In [27]:
##########################################################
##### Correlation coefficient threshold applied here #####
##########################################################
# Pull off users that are highly correlated in ratings
# True would prevent CustomerID index from being retained as a column
corr_df5 = corr_df4.drop(axis=0, index = random_id)
top_users = corr_df5[(corr_df5["corr"] >= rho_thresh)]
top_users.head(10)

,corr
CustomerID,
1180814,0.589639
1726783,0.565858
31387,0.546136
2444333,0.517946
1688893,0.508165
1500471,0.507394
1158943,0.505248
1683765,0.505033
1352424,0.504831


In [28]:
top_users.size

# Number of users left after applying movie-overlap and rating-correlation filters
# Compare to max_value above

301

In [29]:
# Uninteresting reformatting
top_users.reset_index(drop=False, inplace=True)

# Check: format to use isin(): sequential index, two labeled columns
top_users.head(10)

,CustomerID,corr
0,1180814,0.589639
1,1726783,0.565858
2,31387,0.546136
3,2444333,0.517946
4,1688893,0.508165
5,1500471,0.507394
6,1158943,0.505248
7,1683765,0.505033
8,1352424,0.504831
9,454635,0.504631


In [30]:
# top_users['CustomerID'].astype(int)
# similar_users_df['CustomerID'].astype(int)
# top_users.dtypes

In [31]:
# Pull off all highly correleted highly overlapping users from original dataframe with full detail
similar_users_df2 = similar_users_df[similar_users_df['CustomerID'].isin(top_users['CustomerID'])]
similar_users_df2.head(10)


,index,Movie_Id,CustomerID,Rating,Date
626,53625377,9756,7,4,2005-05-23
627,61339957,11182,7,5,2004-06-09
628,8233216,1645,7,4,2005-05-23
629,19688511,3756,7,3,2004-09-27
630,42722880,7586,7,4,2004-09-27
631,26180669,4874,7,5,2005-10-30
632,74113383,13433,7,4,2005-05-23
633,32095365,5843,7,5,2005-05-23
634,30326544,5544,7,3,2005-05-23
635,69606649,12605,7,4,2005-05-23


In [32]:
# Merge in correlation coefficient dictionary, for later credibility weighting of ratings
# merge is preferred to concat for columns
similar_users_df3 = pd.merge(similar_users_df2, top_users, how='inner')
similar_users_df3.head(10)

,index,Movie_Id,CustomerID,Rating,Date,corr
0,53625377,9756,7,4,2005-05-23,0.41326
1,61339957,11182,7,5,2004-06-09,0.41326
2,8233216,1645,7,4,2005-05-23,0.41326
3,19688511,3756,7,3,2004-09-27,0.41326
4,42722880,7586,7,4,2004-09-27,0.41326
5,26180669,4874,7,5,2005-10-30,0.41326
6,74113383,13433,7,4,2005-05-23,0.41326
7,32095365,5843,7,5,2005-05-23,0.41326
8,30326544,5544,7,3,2005-05-23,0.41326
9,69606649,12605,7,4,2005-05-23,0.41326


## __Parameter selection__

The goal is to whittle down the initial 20 million records to about half a million. If the output is far fewer, the filter parameters for correlation and movie count can be lowered and raised respectively and the program restarted *after* the point at which *user_id* is generated.

Note that if *user_id* has rated few movies (less than 100), correlation coefficients can be extreme, either very high (approaching 0.8) or very low (never higher than 0.25).

While high $\rho$ may be misleading in cases of small movie counts, the *relative* values between users are what would used for credibility weighting, i.e., the use of including correclation coefficients in the output file is to differentiate credibility of different users' ratings in predicting *user_id*'s taste.

In [33]:
# Sanity check: sufficient number of ratings left (millions? 20,000 is too low)
similar_users_df3.shape

(261375, 6)

In [34]:
similar_users_df3.sort_values('corr', ascending=False).head()

,index,Movie_Id,CustomerID,Rating,Date,corr
110491,26568745,4951,1180814,4,2004-09-14,0.589639
110241,41389649,7330,1180814,4,2005-06-22,0.589639
110243,41873408,7399,1180814,4,2004-10-19,0.589639
110244,11996678,2340,1180814,3,2005-01-19,0.589639
110245,44753784,7937,1180814,4,2004-10-19,0.589639


In [35]:
top_users

,CustomerID,corr
0,1180814,0.589639
1,1726783,0.565858
2,31387,0.546136
3,2444333,0.517946
4,1688893,0.508165
5,1500471,0.507394
6,1158943,0.505248
7,1683765,0.505033
8,1352424,0.504831
9,454635,0.504631


In [36]:
similar_users_df3.to_csv("../data/rating_corr.csv",header=True, index=False)


#### References:

https://medium.com/codex/hybrid-recommender-system-netflix-prize-dataset-e9f6b4a875aa

https://www.kaggle.com/code/ayseymn/hybrid-recommender-system-netflix/notebook
